This file is part of BrainMolecularAtlas.

Copyright (c) 2021 Blue Brain Project/EPFL 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt

import xlrd
import xlsxwriter

import timeit
import numpy as np
from collections import Counter

from numpy.random import seed
from numpy.random import randint
from scipy.stats import ks_2samp
from scipy.stats import levene
from scipy.stats import fligner
from scipy.stats import kruskal
import scikit_posthocs as sp

import scipy.stats as ss
from scipy.stats import mannwhitneyu


import pickle as pkl



pd.options.display.max_columns = None
pd.options.display.max_rows = None


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
sns.set_style('whitegrid',{'axes.grid':False})

In [3]:
import pickle as pkl

with open('../data/6_df_processed.pkl','rb') as f:
    df = pkl.load(f)
print(len(df))

df["log_conc_uM_medNorm"]  = df["log_conc_uM_medNorm"].astype('float64')



custom_dict = {'Itzhak 2017':0,
               'Wisniewski 2015':1,
              'Bai 2020':2,
              'Hasan 2020':3,
              'Geiger 2013':4,
              'Kjell 2020':5,
              'Fecher 2019':6,
              'Guergues 2019':7,
              'Hamezah 2019':8,
              'McKetney 2019':9,
              'Hamezah 2018':10,
              'Krogager 2018':11,
              'Zhu 2018':12,
              'Carlyle 2017':13,
              'Sharma 2015, isolated':14,
              'Sharma 2015, cultured':15,
              'Han 2014':16,
              'Davis 2019':17,
              'Chuang 2018':18,
              'Fornasiero 2018':19,
              'Hosp 2017, insoluble':20,
              'Hosp 2017, soluble':21,
              'Hosp 2017, CSF':22,
              'Beltran 2016':23,
              'Duda 2018':24}   

df = df.iloc[df['Study'].map(custom_dict).argsort()]


df = df.reset_index(drop=True)

df['exp_conc'] = np.exp(df['log_conc_uM_medNorm'])


2131244


In [4]:
len(df['gene_id_final'].unique())

14700

In [5]:
df.columns

Index(['gene_names', 'Uniprot', 'Study', 'Organism', 'location', 'Age_cat',
       'Age_days', 'condition', 'sample_id', 'molecular_weight_kDa',
       'raw_data', 'raw_data_units', 'gene_name_unified', 'Uniprot_unified',
       'gene_id_final', 'log_raw_data', 'uniprot_from_gn', 'Uniprot_final',
       'TheorPepNum', 'conc_uM', 'log_conc_uM', 'copyNum', 'totalProtein',
       'totalVolume', 'sample_full_id', 'compound_gene_protein_id', 'row_id',
       'row_gene_id', 'gene_id_dd', 'log_conc_uM_medNorm', 'exp_conc'],
      dtype='object')

In [6]:
df4supp = df4supp.reset_index(drop=True)

In [6]:
df4supp = df.copy()
df4supp = df4supp.drop(columns=['Age_days','gene_name_unified', 'Uniprot_unified','log_raw_data','uniprot_from_gn','TheorPepNum','copyNum', 'totalProtein',
       'totalVolume', 'sample_full_id', 'compound_gene_protein_id', 'row_id',
       'row_gene_id', 'exp_conc'])
df4supp.columns


df4supp = df4supp.reset_index(drop=True)


In [8]:
len(df4supp['gene_id_final'].unique())

14700

In [9]:
#df4supp.to_csv("../data/S1Table.txt",sep='\t',index=False)

In [19]:
df.loc[df['condition'].isin(['AD']),'location'].unique()

array(['cortex'], dtype=object)

In [21]:
df.loc[df['condition'].isin(['AD']),'Study'].unique()

array(['Bai 2020'], dtype=object)

In [23]:
df.loc[df['condition'].isin(['AD']),'Organism'].unique()

array(['mouse'], dtype=object)

In [20]:
df.loc[df['condition'].isin(['Alzheimer']),'location'].unique()

array(['cortex', 'striatum', 'hippocampus'], dtype=object)

In [22]:
df.loc[df['condition'].isin(['Alzheimer']),'Study'].unique()

array(['Hamezah 2019'], dtype=object)

In [24]:
df.loc[df['condition'].isin(['Alzheimer']),'Organism'].unique()

array(['mouse'], dtype=object)

In [26]:
df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex'),'condition'].unique()

array(['AD', 'control', 'EAE', nan, 'Alzheimer', 'WT', 'young', 'adult'],
      dtype=object)

In [27]:
df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex'),'Study'].unique()

array(['Bai 2020', 'Hasan 2020', 'Geiger 2013', 'Kjell 2020',
       'Hamezah 2019', 'Sharma 2015, isolated', 'Fornasiero 2018',
       'Hosp 2017, insoluble', 'Hosp 2017, soluble', 'Duda 2018'],
      dtype=object)

In [28]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Geiger 2013'])),'gene_id_final'].unique())

4725

In [30]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Bai 2020'])),'gene_id_final'].unique())

9291

In [31]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Hasan 2020'])),'gene_id_final'].unique())

7435

In [32]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Kjell 2020'])),'gene_id_final'].unique())

6974

In [33]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Hamezah 2019'])),'gene_id_final'].unique())

2752

In [34]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Sharma 2015, isolated'])),'gene_id_final'].unique())

8100

In [35]:
len(df.loc[(df['Organism'].isin(['mouse'])) & (df['location']=='cortex') & \
           (df['Study'].isin(['Fornasiero 2018'])),'gene_id_final'].unique())

1648

In [5]:
#df.to_csv("../data/df4r.txt",sep='\t',index=False)

In [6]:
df2 = df.loc[(df['condition'].isin(['young', 'adult',  'control',  'WT','SORT'])) | (df['condition'].isna())]

df2 = df2.reset_index(drop=True)

In [7]:
#df2.to_csv("../data/df4r_healthy.txt",sep='\t')

In [35]:
df3['Study'].unique()

array(['Itzhak 2017', 'Wisniewski 2015', 'Bai 2020', 'Hasan 2020',
       'Geiger 2013', 'Kjell 2020', 'Fecher 2019', 'Guergues 2019',
       'Hamezah 2019', 'Hamezah 2018', 'Krogager 2018', 'Zhu 2018',
       'Sharma 2015, isolated', 'Sharma 2015, cultured', 'Han 2014',
       'Chuang 2018', 'Fornasiero 2018', 'Hosp 2017, insoluble',
       'Hosp 2017, soluble', 'Hosp 2017, CSF', 'Duda 2018'], dtype=object)

In [8]:
df2['Age_cat'].unique()

array(['embr', 'adult', 'middle-aged', 'mature adult', 'old', 'young',
       'child', 'adolescence', 'post-mortem'], dtype=object)

In [9]:
df3 = df2.loc[df2['Organism'].isin(['mouse','rat'])]
df3 = df3.reset_index(drop=True)

df3['Age_cat_agg'] = None

df3.loc[df3['Age_cat'].isin(['adolescence','child','young']),'Age_cat_agg'] = "young"
df3.loc[df3['Age_cat'].isin(['adult','mature adult']),'Age_cat_agg'] = "adult"
df3.loc[df3['Age_cat'].isin(['middle-aged','old']),'Age_cat_agg'] = "old"


In [10]:
#df3.to_csv("../data/df4r_noH_healthy.txt",sep='\t',index=False)

In [11]:
df3br = df3.loc[(df3['Study']!='Duda 2018') & (df3['location'].isin(['brain','cortex', 'spinal cord', 'striatum', 'cerebellum', 'brainstem','hippocampus', 'midbrain', 'olfactory bulb', 'medulla',
 'corpus callosum', 'optic nerve', 'thalamus','csf']))].copy()


In [12]:
#df3br.to_csv("../data/df4r_br.txt",sep='\t',index=False)

In [13]:
df3cells = df3.loc[(df3['Study']!='Duda 2018') & (df3['location'].isin([ 'microglia', 'neurons','astrocytes','oligodendrocytes']))].copy()

In [14]:
#df3cells.to_csv("../data/df4r_cells.txt",sep='\t',index=False)

In [6]:
df4g = df.copy()

df4g = df4g.drop(columns=['gene_names', 'Uniprot', 
       'Age_days', 'sample_id', 'molecular_weight_kDa',
       'raw_data', 'raw_data_units', 'gene_name_unified', 'Uniprot_unified',
        'log_raw_data', 'uniprot_from_gn', 'Uniprot_final',
       'TheorPepNum', 'conc_uM', 'log_conc_uM', 'copyNum', 'totalProtein',
       'totalVolume', 'sample_full_id', 'compound_gene_protein_id', 'row_id',
       'row_gene_id', 'gene_id_dd', 'exp_conc'])

df4g.columns

Index(['Study', 'Organism', 'location', 'Age_cat', 'condition',
       'gene_id_final', 'log_conc_uM_medNorm'],
      dtype='object')

In [7]:
df4g = df4g.drop(columns=['Organism', 'location', 'Age_cat', 'condition'])
df4g.columns

Index(['Study', 'gene_id_final', 'log_conc_uM_medNorm'], dtype='object')

In [23]:
with open('../data/commonGenesStudiesDfg_28july2020.pkl','rb') as f:
    commonGenesStudiesDfg = pkl.load(f)

In [24]:
np.max(commonGenesStudiesDfg.loc[commonGenesStudiesDfg['numStudies']==5,'numCommonGenes'])

6333

In [52]:
np.max(commonGenesStudiesDfg.loc[commonGenesStudiesDfg['numStudies']==6,'numCommonGenes'])

5735

In [10]:
# find combination of Study with max number of common genes #https://stackoverflow.com/questions/49183330/largest-sub-array-with-no-nan


dfgs = df4g[['Study','gene_id_final','log_conc_uM_medNorm']].groupby(['Study','gene_id_final']).median()
dfgs = df4g.reset_index()


d = {name: dfgs[dfgs['Study']==name] for name in dfgs['Study'].unique()}


gene_st = dict()

for st in df4g['Study'].unique():
    gene_st[st] = df4g[df4g['Study']==st]['gene_id_final'].unique()


c1 = set(list(gene_st.values())[0])
l1 = list()



for gs in list(gene_st.values())[1:len(gene_st.values())]:
    sc1 = set(c1)
    c1 = sc1.intersection(gs)
    
    l1.append(len(c1))

#commonGenesAll = list(set.intersection(*map(set, list(gene_st.values())))) 
#commonGenesAll


import itertools 

# list of arrays to list of lists

ll_gene_st_v = [l.tolist() for l in list(gene_st.values())]


#print(combinationsList(ll_gene_st_v[0:3], 2))

commonGenesAll = list(set.intersection(*map(set, ll_gene_st_v))) 


print(len(ll_gene_st_v))


ll_gene_st_v_sets = [set(l) for l in ll_gene_st_v]

cache = {str(i):s for i,s in enumerate(ll_gene_st_v_sets)}

n = len(ll_gene_st_v_sets)

print(n)

indexes = [i for i in range(n)]

for i in range(2, n):
    indexes_comb = itertools.combinations(indexes, i)
    print(i)
    
    for comb in indexes_comb:
        comb_l = list(comb)
        comb_str = [str(c) for c in comb_l]
        key = " ".join(comb_str)
        prev_key = " ".join(comb_str[0:-1])
        last_set = ll_gene_st_v_sets[comb_l[-1]]
        cache[key] = last_set.intersection(cache[prev_key])


25
25
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24


In [11]:
len(list(cache.keys()))

33554430

In [12]:
cache_keys = list(cache.keys())

In [14]:
commonGenesStudiesNumList = {i: [] for i in range(1, len(ll_gene_st_v))}

for k, v in cache.items():
    commonGenesStudiesNumList[len(k.split(" "))].append(len(v))

print(np.max(commonGenesStudiesNumList[1]))

10413


In [15]:
print(np.max(commonGenesStudiesNumList[24]))

72


In [16]:
print(np.max(commonGenesStudiesNumList[15]))

1643


In [18]:
print(np.max(commonGenesStudiesNumList[8]))

4648


In [19]:
# get list of needed studies

for k, v in cache.items():
    if (len(k.split(" ")) == 24) & (len(v) == 72):
        print(k)
        

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23


In [17]:
# get list of needed studies

for k, v in cache.items():
    if (len(k.split(" ")) == 8) & (len(v) == 4648):
        print(k)
        

0 2 3 5 9 14 15 21


In [18]:
len(ll_gene_st_v_sets)

25

In [20]:
df4g['Study'].unique()[[0,2,3,5,9,14,15,21]]

array(['Itzhak 2017', 'Bai 2020', 'Hasan 2020', 'Kjell 2020',
       'McKetney 2019', 'Sharma 2015, isolated', 'Sharma 2015, cultured',
       'Hosp 2017, soluble'], dtype=object)

In [22]:
gene_in_8_studies = cache["0 2 3 5 9 14 15 21"]


with open('gene_in_8_studies_30july2020.pkl', 'wb') as fp:
    pkl.dump(gene_in_8_studies, fp)

In [23]:
df_8_studies = df4g.loc[(df4g['Study'].isin(df4g['Study'].unique()[[0,2,3,5,9,14,15,21]]) ) & \
                        (df4g['gene_id_final'].isin(gene_in_8_studies))].copy()
print(len(df_8_studies))

df_8_studies.head()
#df_8_studies.to_csv("../data/df_8_studies.txt",sep='\t',index=False)

854474


,Study,gene_id_final,log_conc_uM_medNorm
0,Itzhak 2017,LIG3,0.149491
1,Itzhak 2017,PTEN,-1.627674
3,Itzhak 2017,SMG1,-3.162047
4,Itzhak 2017,PRPF38B,-0.174327
5,Itzhak 2017,ATP6V1G1,1.029038


In [ ]:
#df_8_studies.to_csv("../data/df_8_studies.txt",sep='\t',index=False)

In [20]:
dfbr = df.loc[(df['Study']!='Duda 2018') & (df['location'].isin(['brain','cortex', 'striatum', 'cerebellum', 'brainstem','hippocampus', 'midbrain', 'olfactory bulb', 'medulla', 'thalamus'])) &\
             ((df['condition'].isin(['young', 'adult',  'control',  'WT','SORT'])) | (df['condition'].isna()))].copy()
dfbr = dfbr.loc[:,['gene_id_final','location','log_conc_uM_medNorm']]
dfbr = dfbr.reset_index(drop=True)

In [21]:
# find combination of location with max number of common genes

dfgs = dfbr[['location','gene_id_final','log_conc_uM_medNorm']].groupby(['location','gene_id_final']).median()
dfgs = dfbr.reset_index()


d = {name: dfgs[dfgs['location']==name] for name in dfgs['location'].unique()}


gene_st = dict()

for st in dfbr['location'].unique():
    gene_st[st] = dfbr[dfbr['location']==st]['gene_id_final'].unique()


c1 = set(list(gene_st.values())[0])
l1 = list()



for gs in list(gene_st.values())[1:len(gene_st.values())]:
    sc1 = set(c1)
    c1 = sc1.intersection(gs)
    
    l1.append(len(c1))

#commonGenesAll = list(set.intersection(*map(set, list(gene_st.values())))) 
#commonGenesAll


import itertools 

# list of arrays to list of lists

ll_gene_st_v = [l.tolist() for l in list(gene_st.values())]


#print(combinationsList(ll_gene_st_v[0:3], 2))

commonGenesAll = list(set.intersection(*map(set, ll_gene_st_v))) 


print(len(ll_gene_st_v))


ll_gene_st_v_sets = [set(l) for l in ll_gene_st_v]

cache = {str(i):s for i,s in enumerate(ll_gene_st_v_sets)}

n = len(ll_gene_st_v_sets)

print(n)

indexes = [i for i in range(n)]

for i in range(2, n):
    indexes_comb = itertools.combinations(indexes, i)
    print(i)
    
    for comb in indexes_comb:
        comb_l = list(comb)
        comb_str = [str(c) for c in comb_l]
        key = " ".join(comb_str)
        prev_key = " ".join(comb_str[0:-1])
        last_set = ll_gene_st_v_sets[comb_l[-1]]
        cache[key] = last_set.intersection(cache[prev_key])


10
10
2
3
4
5
6
7
8
9


In [22]:
commonGenesStudiesNumList = {i: [] for i in range(1, len(ll_gene_st_v))}

for k, v in cache.items():
    commonGenesStudiesNumList[len(k.split(" "))].append(len(v))

print(np.max(commonGenesStudiesNumList[9]))

4343


In [24]:
# get list of needed locations

for k, v in cache.items():
    if (len(k.split(" ")) == 9) & (len(v) == 4343):
        print(k)
        

0 1 2 3 4 5 7 8 9


In [25]:
gene_in_br = cache["0 1 2 3 4 5 7 8 9"]


with open('gene_in_br_30july2020.pkl', 'wb') as fp:
    pkl.dump(gene_in_br, fp)

In [26]:
df_br = dfbr.loc[(dfbr['location'].isin(dfbr['location'].unique()[[0, 1, 2, 3, 4, 5, 7, 8, 9]]) ) & \
                        (dfbr['gene_id_final'].isin(gene_in_br))].copy()
print(len(df_br))

df_br.to_csv("../data/df_br_30july2020.txt",sep='\t',index=False)

df_br.head()


676034


,gene_id_final,location,log_conc_uM_medNorm
2,GNAS,brain,-0.824751
3,MVK,brain,-3.765348
4,METAP1,brain,-2.759008
5,PDIA3,brain,2.297793
6,NPEPPS,brain,1.288316


In [27]:
df['location'].unique()

array(['nucleus', 'subcellular not specified', 'large protein complex',
       'plasma membrane', 'er', 'golgi', 'actin binding proteins',
       'mitochondria', 'endosome', 'peroxisome', 'lysosome', 'brain',
       'cortex', 'spinal cord', 'striatum', 'hippocampus', 'cerebellum',
       'brainstem', 'midbrain', 'olfactory bulb', 'medulla',
       'subependymal zone', 'medial subependymal zone', 'microglia',
       'amygdala', 'thalamus', 'neurons', 'corpus callosum',
       'optic nerve', 'astrocytes', 'oligodendrocytes', 'axon', 'csf',
       'cytoplasm'], dtype=object)

In [28]:
dfcell = df.loc[(df['Study']!='Duda 2018') & (df['location'].isin(['neurons','astrocytes', 'oligodendrocytes', 'microglia'])) &\
             ((df['condition'].isin(['young', 'adult',  'control',  'WT','SORT'])) | (df['condition'].isna()))].copy()
dfcell = dfcell.loc[:,['gene_id_final','location','log_conc_uM_medNorm']]
dfcell = dfcell.reset_index(drop=True)

In [29]:
# find combination of location with max number of common genes

dfgs = dfcell[['location','gene_id_final','log_conc_uM_medNorm']].groupby(['location','gene_id_final']).median()
dfgs = dfcell.reset_index()


d = {name: dfgs[dfgs['location']==name] for name in dfgs['location'].unique()}


gene_st = dict()

for st in dfcell['location'].unique():
    gene_st[st] = dfcell[dfcell['location']==st]['gene_id_final'].unique()


c1 = set(list(gene_st.values())[0])
l1 = list()



for gs in list(gene_st.values())[1:len(gene_st.values())]:
    sc1 = set(c1)
    c1 = sc1.intersection(gs)
    
    l1.append(len(c1))

import itertools 

# list of arrays to list of lists

ll_gene_st_v = [l.tolist() for l in list(gene_st.values())]

commonGenesAll = list(set.intersection(*map(set, ll_gene_st_v))) 

print(len(ll_gene_st_v))

ll_gene_st_v_sets = [set(l) for l in ll_gene_st_v]

cache = {str(i):s for i,s in enumerate(ll_gene_st_v_sets)}

n = len(ll_gene_st_v_sets)

print(n)

indexes = [i for i in range(n)]

for i in range(2, n):
    indexes_comb = itertools.combinations(indexes, i)
    print(i)
    
    for comb in indexes_comb:
        comb_l = list(comb)
        comb_str = [str(c) for c in comb_l]
        key = " ".join(comb_str)
        prev_key = " ".join(comb_str[0:-1])
        last_set = ll_gene_st_v_sets[comb_l[-1]]
        cache[key] = last_set.intersection(cache[prev_key])


4
4
2
3


In [32]:
commonGenesStudiesNumList = {i: [] for i in range(1, len(ll_gene_st_v))}

for k, v in cache.items():
    commonGenesStudiesNumList[len(k.split(" "))].append(len(v))

print(np.max(commonGenesStudiesNumList[3]))

8313


In [36]:
len(commonGenesAll)

7940

In [24]:
# get list of needed locations

for k, v in cache.items():
    if (len(k.split(" ")) == 9) & (len(v) == 4343):
        print(k)
        

0 1 2 3 4 5 7 8 9


In [34]:
with open('gene_in_dfcell_30july2020.pkl', 'wb') as fp:
    pkl.dump(commonGenesAll, fp)

In [35]:
df_cell = dfcell.loc[(dfcell['location'].isin(['neurons','astrocytes', 'oligodendrocytes', 'microglia']) ) & \
                        (dfcell['gene_id_final'].isin(commonGenesAll))].copy()
print(len(df_cell))

df_cell.to_csv("../data/df_cell_30july2020.txt",sep='\t',index=False)

df_cell.head()


286985


,gene_id_final,location,log_conc_uM_medNorm
0,DDX49,microglia,-2.580261
1,NAP1L1,microglia,1.046464
2,MYADM,microglia,-2.851195
3,MYADM,microglia,-3.047748
4,ELP3,microglia,-2.843318


In [25]:
full_df_8_studies = df.loc[(df['Study'].isin(df4g['Study'].unique()[[0,2,3,5,9,14,15,21]]) ) & \
                        (df['gene_id_final'].isin(gene_in_8_studies))].copy()
print(len(full_df_8_studies))

full_df_8_studies.head()

854474


,gene_names,Uniprot,Study,Organism,location,Age_cat,Age_days,condition,sample_id,molecular_weight_kDa,raw_data,raw_data_units,gene_name_unified,Uniprot_unified,gene_id_final,log_raw_data,uniprot_from_gn,Uniprot_final,TheorPepNum,conc_uM,log_conc_uM,copyNum,totalProtein,totalVolume,sample_full_id,compound_gene_protein_id,row_id,row_gene_id,gene_id_dd,log_conc_uM_medNorm,exp_conc
0,LIG3,Q80ZH7;P97386;B1AT03;K4DI59;P97386-2,Itzhak 2017,mouse,nucleus,embr,15,NaN,NaN,112.78,347.264454,Median cellular concentration [nM],LIG3,P97386,LIG3,5.850087,"[Q6UXM1, P49916, P97386]",P97386,None,0.347264,-1.057669,None,None,None,Itzhak 2017_mouse_nucleus_15_nan_nan,LIG3#P97386,LIG3#P97386@Itzhak 2017_mouse_nucleus_15_nan_nan,LIG3@Itzhak 2017_mouse_nucleus_15_nan_nan,None,0.149491,1.161243
1,PTEN,O08586,Itzhak 2017,mouse,subcellular not specified,embr,15,NaN,NaN,47.152,58.728304,Median cellular concentration [nM],PTEN,O08586,PTEN,4.072922,"[O08586, P60484]",O08586,None,0.0587283,-2.834833,None,None,None,Itzhak 2017_mouse_subcellular not specified_15...,PTEN#O08586,PTEN#O08586@Itzhak 2017_mouse_subcellular not ...,PTEN@Itzhak 2017_mouse_subcellular not specifi...,None,-1.627674,0.196386
3,SMG1,Q8BKX6;J3QQ22,Itzhak 2017,mouse,large protein complex,embr,15,NaN,NaN,409.76,12.661291,Median cellular concentration [nM],SMG1,Q8BKX6,SMG1,2.538549,"[Q96Q15, Q8BKX6]",Q8BKX6,None,0.0126613,-4.369206,None,None,None,Itzhak 2017_mouse_large protein complex_15_nan...,SMG1#Q8BKX6,SMG1#Q8BKX6@Itzhak 2017_mouse_large protein co...,SMG1@Itzhak 2017_mouse_large protein complex_1...,None,-3.162047,0.042339
4,PRPF38B,Q80SY5,Itzhak 2017,mouse,large protein complex,embr,15,NaN,NaN,63.752,251.204977,Median cellular concentration [nM],PRPF38B,Q80SY5,PRPF38B,5.526269,"[Q6AXY7, Q80SY5, Q5VTL8]",Q80SY5,None,0.251205,-1.381486,None,None,None,Itzhak 2017_mouse_large protein complex_15_nan...,PRPF38B#Q80SY5,PRPF38B#Q80SY5@Itzhak 2017_mouse_large protein...,PRPF38B@Itzhak 2017_mouse_large protein comple...,None,-0.174327,0.840022
5,ATP6V1G1,Q9CR51;Q5HZY7,Itzhak 2017,mouse,large protein complex,embr,15,NaN,NaN,13.724,836.840934,Median cellular concentration [nM],ATP6V1G1,Q9CR51,ATP6V1G1,6.729634,"[O75348, Q9CR51]",Q9CR51,None,0.836841,-0.178121,None,None,None,Itzhak 2017_mouse_large protein complex_15_nan...,ATP6V1G1#Q9CR51,ATP6V1G1#Q9CR51@Itzhak 2017_mouse_large protei...,ATP6V1G1@Itzhak 2017_mouse_large protein compl...,None,1.029038,2.798373


In [ ]:
full_df_8_studies = full_df_8_studies.drop(columns=['gene_names', 'Uniprot', 
       'Age_days', 'sample_id', 'molecular_weight_kDa',
       'raw_data', 'raw_data_units', 'gene_name_unified', 'Uniprot_unified',
        'log_raw_data', 'uniprot_from_gn', 'Uniprot_final',
       'TheorPepNum', 'conc_uM', 'log_conc_uM', 'copyNum', 'totalProtein',
       'totalVolume', 'sample_full_id', 'compound_gene_protein_id', 'row_id',
       'row_gene_id', 'gene_id_dd', 'exp_conc'])

full_df_8_studies.columns

In [ ]:
#full_df_8_studies.to_csv("../data/full_df_8_studies.txt",sep='\t',index=False)

In [26]:
# no Itzhak because it's subcell

full_df_8_studies = df.loc[(df['Study'].isin(df4g['Study'].unique()[[2,3,5,9,14,15,21]]) ) & \
                        (df['gene_id_final'].isin(gene_in_8_studies))].copy()
print(len(full_df_8_studies))

full_df_8_studies = full_df_8_studies.drop(columns=['gene_names', 'Uniprot', 
       'Age_days', 'sample_id', 'molecular_weight_kDa',
       'raw_data', 'raw_data_units', 'gene_name_unified', 'Uniprot_unified',
        'log_raw_data', 'uniprot_from_gn', 'Uniprot_final',
       'TheorPepNum', 'conc_uM', 'log_conc_uM', 'copyNum', 'totalProtein',
       'totalVolume', 'sample_full_id', 'compound_gene_protein_id', 'row_id',
       'row_gene_id', 'gene_id_dd', 'exp_conc'])

full_df_8_studies.columns

849508


Index(['Study', 'Organism', 'location', 'Age_cat', 'condition',
       'gene_id_final', 'log_conc_uM_medNorm'],
      dtype='object')

In [27]:
full_df_8_studies['sample'] = full_df_8_studies['Study'].astype(str)+full_df_8_studies['location'].astype(str) #+full_df_8_studies['Age_cat'].astype(str)+full_df_8_studies['condition'].astype(str)  


In [64]:
full_df_8_studies.head()

,Study,Organism,location,Age_cat,condition,gene_id_final,log_conc_uM_medNorm,sample
28527,Bai 2020,mouse,cortex,mature adult,AD,MCAT,-2.137187,Bai 2020cortex
28529,Bai 2020,mouse,cortex,mature adult,AD,VDAC3,3.163652,Bai 2020cortex
28531,Bai 2020,mouse,cortex,adult,AD,DIRAS1,0.750658,Bai 2020cortex
28533,Bai 2020,mouse,cortex,mature adult,AD,TNFRSF21,-2.562858,Bai 2020cortex
28535,Bai 2020,mouse,cortex,middle-aged,control,KCTD4,-1.666867,Bai 2020cortex


In [ ]:
len(full_df_8_studies['sample'].unique())

In [97]:
# find combination of sample with max number of common genes

dfgs = full_df_8_studies[['sample','gene_id_final','log_conc_uM_medNorm']].groupby(['sample','gene_id_final']).median()
dfgs = dfgs.reset_index()


d = {name: dfgs[dfgs['sample']==name] for name in dfgs['sample'].unique()}
print("d ",len(d))

gene_st = dict()

for st in full_df_8_studies['sample'].unique():
    gene_st[st] = full_df_8_studies[full_df_8_studies['sample']==st]['gene_id_final'].unique()


c1 = set(list(gene_st.values())[0])
l1 = list()



for gs in list(gene_st.values())[1:len(gene_st.values())]:
    sc1 = set(c1)
    c1 = sc1.intersection(gs)
    
    l1.append(len(c1))

#commonGenesAll = list(set.intersection(*map(set, list(gene_st.values())))) 
#commonGenesAll


import itertools 

# list of arrays to list of lists

ll_gene_st_v = [l.tolist() for l in list(gene_st.values())]


#print(combinationsList(ll_gene_st_v[0:3], 2))

commonGenesAll = list(set.intersection(*map(set, ll_gene_st_v))) 


print(len(ll_gene_st_v))


ll_gene_st_v_sets = [set(l) for l in ll_gene_st_v]

cache = {str(i):s for i,s in enumerate(ll_gene_st_v_sets)}

print(len(list(cache.keys())))

n = len(ll_gene_st_v_sets)
print("n: ",n)

indexes = [i for i in range(n)]


d  38
38
38
n:  38


In [ ]:
for i in range(2, n):
    indexes_comb = itertools.combinations(indexes, i)
    print(i)

    for comb in indexes_comb:
        
        comb_l = list(comb)
        
        #print(comb_l)
        
        comb_str = [str(c) for c in comb_l]
        key = " ".join(comb_str)
        prev_key = " ".join(comb_str[0:-1])
        last_set = ll_gene_st_v_sets[comb_l[-1]]
        cache[key] = last_set.intersection(cache[prev_key])  # due to this I have to run for samples of 2,3... may be improved


2
3
4
5
6


In [89]:
len(list(cache.keys()))

22433

In [33]:
commonGenesStudiesNumList = {i: [] for i in range(1, len(ll_gene_st_v))}


for k, v in cache.items():
    commonGenesStudiesNumList[len(k.split(" "))].append(len(v))

In [37]:
len(cache.items())

79

In [ ]:
# further figures using this data are in R code